In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Cargar los datos desde los archivos CSV
movies_df = pd.read_csv('../data/rotten_tomatoes_movies.csv')  # Carga la información de las películas
reviews_df = pd.read_csv('../data/rotten_tomatoes_critic_reviews.csv')  # Carga la información de las críticas

In [4]:
# Verificar que los datos se han cargado correctamente
# Muestra las primeras filas de ambos dataframes para inspeccionar la estructura de los datos
print("Primeras filas de las películas:")
display(movies_df.head())  # Muestra las primeras filas de las películas
print("\nPrimeras filas de las críticas:")
display(reviews_df.head())  # Muestra las primeras filas de las críticas

Primeras filas de las películas:


,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3



Primeras filas de las críticas:


,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [5]:
# Preprocesamiento de datos en las críticas
# Convertir el texto a minúsculas
reviews_df['review_content'] = reviews_df['review_content'].str.lower()  # Cambiar todo el texto a minúsculas

# Verificación del cambio
print("\nPrimeras filas después de convertir a minúsculas:")
display(reviews_df[['review_content']].head())  # Verificar que se ha convertido correctamente



Primeras filas después de convertir a minúsculas:


,review_content
0,a fantasy adventure that fuses greek mythology...
1,"uma thurman as medusa, the gorgon with a coiff..."
2,with a top-notch cast and dazzling special eff...
3,whether audiences will get behind the lightnin...
4,what's really lacking in the lightning thief i...


In [6]:
# Limpiar el texto (eliminar signos de puntuación y caracteres no deseados)
def clean_text(text):
    if isinstance(text, str):  # Verificamos si el texto es una cadena de caracteres
        return re.sub(r'[^\w\s]', '', text)  # Solo dejar letras y números
    return ''  # Si no es un texto, devolver una cadena vacía

# Aplicar la limpieza de texto a las críticas
reviews_df['review_content'] = reviews_df['review_content'].apply(clean_text)
# Verificación de la limpieza
print("\nPrimeras filas después de limpiar el texto:")
display(reviews_df[['review_content']].head())  # Verificar que se ha limpiado correctamente


Primeras filas después de limpiar el texto:


,review_content
0,a fantasy adventure that fuses greek mythology...
1,uma thurman as medusa the gorgon with a coiffu...
2,with a topnotch cast and dazzling special effe...
3,whether audiences will get behind the lightnin...
4,whats really lacking in the lightning thief is...


In [7]:
# Tokenización
reviews_df['tokens'] = reviews_df['review_content'].apply(word_tokenize)  # Convertir el texto en una lista de palabras (tokens)

# Verificación de la tokenización
print("\nPrimeras filas después de tokenizar el texto:")
display(reviews_df[['tokens']].head())  # Verificar que las críticas han sido tokenizadas correctamente


Primeras filas después de tokenizar el texto:


,tokens
0,"[a, fantasy, adventure, that, fuses, greek, my..."
1,"[uma, thurman, as, medusa, the, gorgon, with, ..."
2,"[with, a, topnotch, cast, and, dazzling, speci..."
3,"[whether, audiences, will, get, behind, the, l..."
4,"[whats, really, lacking, in, the, lightning, t..."


In [8]:
# Eliminar stop words y aplicar stemming
stop_words = set(stopwords.words('english'))  # Lista de stop words en inglés
stemmer = PorterStemmer()  # Inicializar el stemmer para reducción de palabras a su raíz

def clean_tokens(tokens):
    return [stemmer.stem(word) for word in tokens if word not in stop_words]  # Filtrar stop words y aplicar stemming

# Aplicar la limpieza de tokens a las críticas
reviews_df['clean_tokens'] = reviews_df['tokens'].apply(clean_tokens)

# Verificación de la eliminación de stop words y aplicación de stemming
print("\nPrimeras filas después de eliminar stop words y aplicar stemming:")
display(reviews_df[['clean_tokens']].head())  # Verificar que las críticas han sido procesadas correctamente


Primeras filas después de eliminar stop words y aplicar stemming:


,clean_tokens
0,"[fantasi, adventur, fuse, greek, mytholog, con..."
1,"[uma, thurman, medusa, gorgon, coiffur, writh,..."
2,"[topnotch, cast, dazzl, special, effect, tide,..."
3,"[whether, audienc, get, behind, lightn, thief,..."
4,"[what, realli, lack, lightn, thief, genuin, se..."


In [9]:
# Crear el modelo TF-IDF para representar las críticas
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df['review_content'])  # Transformar el texto limpio en vectores TF-IDF

# Verificación del modelo TF-IDF
print("\nForma de la matriz TF-IDF:")
print(X.shape)  # Mostrar el tamaño de la matriz, que debe ser (número de críticas, número de términos)


Forma de la matriz TF-IDF:
(1130017, 220040)


In [10]:
# Preprocesar la consulta de la misma manera que las críticas
def preprocess_query(query):
    # Convertir la consulta a minúsculas
    query = query.lower()
    
    # Limpiar el texto (eliminar caracteres no deseados)
    query = clean_text(query)
    
    # Tokenizar la consulta
    query_tokens = word_tokenize(query)
    
    # Eliminar stop words y aplicar stemming
    query_tokens = clean_tokens(query_tokens)
    
    # Unir los tokens nuevamente para tener el texto limpio para la consulta
    query_cleaned = " ".join(query_tokens)
    
    return query_cleaned

In [11]:
# Consultar el TF-IDF de la consulta
def query_search(query, vectorizer, X, reviews_df, movies_df):
    # Preprocesar la consulta
    query_cleaned = preprocess_query(query)
    
    # Transformar la consulta en el formato de TF-IDF
    query_vector = vectorizer.transform([query_cleaned])
    
    # Calcular la similitud coseno entre la consulta y las críticas
    similarity_scores = np.dot(X, query_vector.T).toarray().flatten()
    
    # Obtener los índices de las críticas relevantes
    relevant_indices = np.argsort(similarity_scores)[::-1][:10]
    
    # Obtener los enlaces de las críticas relevantes
    relevant_critic_links = reviews_df.iloc[relevant_indices]['rotten_tomatoes_link'].values
    
    # Asegurarnos de que los índices no estén fuera de rango
    print(f"Índices relevantes: {relevant_indices}")
    print(f"Links de las críticas relevantes (primeros 10): {relevant_critic_links}")
    
    # Mostrar las películas y enlaces de las críticas relevantes
    print(f"Consulta preprocesada: {query_cleaned}")
    print(f"Vocabulario común entre la consulta y el corpus: {set(query_cleaned.split())}")
    print(f"Similitud para la consulta '{query}':\n{similarity_scores[relevant_indices]}")
    
    # Ahora, busca las películas correspondientes a esos enlaces
    for idx in range(len(relevant_critic_links)):
        movie_link = relevant_critic_links[idx]
        movie_row = movies_df[movies_df['rotten_tomatoes_link'] == movie_link].iloc[0]
        print(f"Relevancia: {similarity_scores[relevant_indices[idx]]:.4f} - Película: {movie_row['movie_title']} - Link: {movie_row['rotten_tomatoes_link']}")

# Llamar a la función con la consulta
query = "space travel"
query_search(query, vectorizer, X, reviews_df, movies_df)


Índices relevantes: [ 346898  110181  141461  464911  349736  844909  844891  110398  110262
 1042501]
Links de las críticas relevantes (primeros 10): ['m/first_man' 'm/ad_astra' 'm/apollo_11_2019'
 'm/it_the_terror_from_beyond_space' 'm/flight_of_the_navigator'
 'm/source_code' 'm/source_code' 'm/ad_astra' 'm/ad_astra'
 'm/time_travelers_wife']
Consulta preprocesada: space travel
Vocabulario común entre la consulta y el corpus: {'travel', 'space'}
Similitud para la consulta 'space travel':
[0.67022824 0.5436185  0.5231926  0.51269205 0.48614755 0.47516708
 0.47516708 0.47434943 0.4697501  0.4567627 ]
Relevancia: 0.6702 - Película: First Man - Link: m/first_man
Relevancia: 0.5436 - Película: Ad Astra - Link: m/ad_astra
Relevancia: 0.5232 - Película: Apollo 11 - Link: m/apollo_11_2019
Relevancia: 0.5127 - Película: It! The Terror From Beyond Space - Link: m/it_the_terror_from_beyond_space
Relevancia: 0.4861 - Película: Flight of the Navigator - Link: m/flight_of_the_navigator
Relevancia